In [163]:
from math import floor

class field(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
        self.generated_terms = []
        self.connected = [] #list of connected terms, includes repeats
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) #+ '\n' \
            #+ 'connected: ' + str(self.connected)
    def __eq__(self, other):
        eq = (self.elementary_str == other.elementary_str) & (self.contractions == other.contractions)
        return eq
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    def get_connected(self):
        return self.connected
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field1 = field(['P'],[], 'P')
                self.elementary.append(field1)
            elif symbol == 'DP':
                field1_deriv = field(['DP'],[], 'DP')
                self.elementary.append(field1_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    def set_connected(self, connected_list):
        self.connected = connected_list
    
    
    def print_connected(self):
        for term in self.get_connected():
            print(term)
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        #print("counter1: " + str(counter1))
        #print("counter2: " + str(counter2))
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    def bubbleSort(self, field_list):
        n = len(field_list)
 
        # Traverse through all array elements
        for i in range(n):
 
            # Last i elements are already in place
            for j in range(0, n-i-1):
 
                # traverse the array from 0 to n-i-1
                # Swap if the element found is greater
                # than the next element
                if field_list[j].massDim() > field_list[j+1].massDim():
                    field_list[j], field_list[j+1] = field_list[j+1], field_list[j]
    
        return field_list
    
    
    def multiply(self, other, symbol_prod=''):
        '''
        Multiplies without contraction
        INPUT:
        self: field object
        other: field object
        symbol_prod: symbol for new field obj output
        
        OUTPUT:
        prod: field object, concatenating self and other, by inserting [0,0] in contractions
        
        '''
        if (not self.get_elementary_str()) or (not other.get_elementary_str()):
            #print("one factor is const")
            contractions_prod = self.get_contractions() + other.get_contractions()
        else:
            contractions_prod = self.get_contractions() + [[0,0]] + other.get_contractions()
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        connected_prod = self.get_connected() + other.get_connected()
        #Sort connected_prod by mass dimension, with lowest first and highest last
        connected_prod_sorted = field.bubbleSort(self, connected_prod)
        
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        prod.set_connected(connected_prod_sorted)
        
        return prod
    
    
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate field terms
        
        OUTPUT
        terms: list of field terms of mass dimension no larger than d.  
        '''
        
        const = field([],[],'1')
        
        terms = [const]
        
        #print("in generate_connected")
        
        if d >= 2:
            #print("in d>=2")
            field1 = field(['P'],[],'P')
            field2 = field(['P'],[],'P')
            prod = field.contract(field1, field2, [0,1], 'PP')
            terms.append(prod)
        if d >= 4:
            deriv1 = field(['DP'],[],'DP')
            deriv2 = field(['DP'],[],'DP')
            prod = field.contract(deriv1, deriv2, [1,1], 'DPDP')
            terms.append(prod)
        
        field1 = field(['P'],[],'P')
        deriv1 = field(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = field.contract(field1, deriv1, [0,1], 'PDP')
        
        while root.massDim() <= d:
            #print("")
            #print("root: ")
            #print(root)
            #print("")
            if root.LORrank() != 0:
                #print("in root.LORrank()!=0")
                deriv1 = field(['DP'],[],'DP')
                root = field.contract(root, deriv1, [1,0], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")
            
            if root.ONrank() != 0:
                #print("in root.ONrank()!=0")
                field1 = field(['P'],[],'P')
                deriv1 = field(['DP'],[],'DP')
                
                new = field.contract(root, field1, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    #print("in new term added")
                    terms.append(new)
                
                root = field.contract(root, deriv1, [0,1], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")    
        
        return terms
    
   
    
    def generate(self, D):
        if D < 2:
            print("D should be an integer >= 2.")
            return []
        else:
            #initiate list of generated terms
            generated = []
            
            #build list of connected terms
            connected_terms_D = self.generate_connected(D)
            connected_len = len(connected_terms_D)
            
            #initiate cache
            const = field([],[],'')
            const.set_connected([field([],[],'')])
            print("const.get_connected(): " + str(const.get_connected()))
            cache = [const]
            #cache = cache + connected_terms_D
            
            
            #set_connected for existing elements in cache
            #for i in range(len(cache)):
                #cache[i].set_connected(connected_terms_D)
            
            #cache_len = len(cache)
            
            
            
            a = 1
            
            print("a = " + str(a))
            print("len(cache): " + str(len(cache)))
            print("")
            
            while a <= floor(D/2):
                
                for j in range(len(cache)):
                    for i in range(connected_len):
                        #print("connected_terms_D[i].massDim(): " + str(connected_terms_D[i].massDim()))
                        #print("cache[j].get_connected(): " + str(type(cache[j].get_connected())))
                        #print("")
                
                        if connected_terms_D[i].massDim() <= cache[j].get_connected()[0].massDim():
                            prod = field.multiply(connected_terms_D[i], cache[j])
                            cache.append(prod)
                
                print("a = " + str(a))
                print("len(cache): " + str(len(cache)))
                print("")
                
                a += 1
           
            
                
            #fill generated list with terms from cache below the masDim cutoff
            for term in cache:
                if term.massDim() <= D:
                    generated.append(term)
                
            return generated
                    
            
            
            
    
    
            
                    
            
    
                
                
            
        


In [164]:
A = field([],[],'')

In [116]:
connected_list = A.generate_connected(10)

In [89]:
A.set_connected(connected_list)

In [90]:
A.get_connected()

In [91]:
for i in _:
    print(i)

elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0


In [92]:
A = field(['P','DP','DP','P'], [[0,1],[1,0],[0,1]], 'A')

In [93]:
A.set_elementary()

In [94]:
B = field(['DP','DP'], [[1,1]], 'B')

In [95]:
B.set_elementary()

In [96]:
fields = [A,B]

In [97]:
fields_sorted = A.bubbleSort(fields)

In [98]:
fields_sorted

[<__main__.field at 0x106c495f8>, <__main__.field at 0x106c49fd0>]

In [99]:
print(fields_sorted[0])

elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: B
massDim: 4
LORrank: 0
ONrank: 0


In [100]:
print(fields_sorted[1])

elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: A
massDim: 6
LORrank: 0
ONrank: 0


In [165]:
generated = A.generate(6)

const.get_connected(): [<__main__.field object at 0x106ce4a58>]
a = 1
len(cache): 1

a = 1
len(cache): 2

a = 2
len(cache): 4

a = 3
len(cache): 8



In [155]:
len(generated)

32

In [139]:
type(A.get_connected())

list

In [153]:
for term in generated:
    print("NEW GENERATED TERM")
    field.print_connected(term)
    print("")

NEW GENERATED TERM
elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0

NEW GENERATED TERM
elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0

NEW GENERATED TERM
elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank

ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0

